In [1]:
import tensorflow as tf
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [2]:
model = tf.keras.models.load_model('classifier.h5')
quantizer = vitis_quantize.VitisQuantizer(model)

In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [4]:

quantized_model = quantizer.quantize_model(calib_dataset = test_set)

[INFO] Start CrossLayerEqualization...
10/10 [==============================] - 1s 82ms/step
[INFO] CrossLayerEqualization Done.
[INFO] Start Quantize Calibration...
63/63 [==============================] - 71s 1s/step
[INFO] Quantize Calibration Done.
[INFO] Start Generating Quantized Model...
[Warning] Skip quantize pos adjustment for layer quant_dense_1, its quantize pos is [i=None, w=8.0, b=8.0, o=3.0]
[INFO] Generating Quantized Model Done.


In [5]:
Quant_model_path = 'vitis-ai-quantized-cat_dog-classification-model.h5'

In [6]:
# Save the model
quantized_model.save(Quant_model_path)

In [7]:
from tensorflow_model_optimization.quantization.keras import vitis_quantize
with vitis_quantize.quantize_scope():
    quantized_model = tf.keras.models.load_model(Quant_model_path)
quantized_model

In [8]:
quantized_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quant_conv2d_1_input (Quanti (None, 64, 64, 3)         4         
_________________________________________________________________
quant_conv2d_1 (QuantizeWrap (None, 62, 62, 32)        906       
_________________________________________________________________
quant_max_pooling2d_1 (Quant (None, 31, 31, 32)        4         
_________________________________________________________________
quant_conv2d_2 (QuantizeWrap (None, 29, 29, 32)        9258      
_________________________________________________________________
quant_max_pooling2d_2 (Quant (None, 14, 14, 32)        4         
_________________________________________________________________
quant_flatten_1 (QuantizeWra (None, 6272)              1         
_________________________________________________________________
quant_dense_1 (QuantizeWrapp (None, 128)               8

In [9]:
!vai_c_tensorflow2 -h

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
usage: vai_c_tensorflow2 [-h] [-m MODEL] [-a ARCH] [-o OUTPUT_DIR]
                         [-n NET_NAME] [-e OPTIONS]

optional arguments:
  -h, --help            show this help message and exit
  -m MODEL, --model MODEL
                        h5 model file
  -a ARCH, --arch ARCH  json file
  -o OUTPUT_DIR, --output_dir OUTPUT_DIR
                        output directory
  -n NET_NAME, --net_name NET_NAME
                        prefix-name for the outputs
  -e OPTIONS, --options OPTIONS
                        extra options


In [10]:
!vai_c_tensorflow2 -m vitis-ai-quantized-cat_dog-classification-model.h5 -a /opt/vitis_ai/compiler/arch/DPUCADX8G/ALVEO/arch.json -o vitis-compiled/ -n cat_dog_XIR

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
ERROR: NO FRONT END SPECIFIED
